# Concept tagging with Weighted Finite State Machines

In [ ]:
from conll import evaluate
import pandas as pd
import os
from utils import *
from pre_process_data import *

Preprocess data

In [ ]:
norm_data_input("dataset/NL2SparQL4NLU.train_no_stop_word.utterances.txt",
                "dataset/NL2SparQL4NLU.train_norm_all_words_no_stop_word.utterances.txt")
norm_data_input("dataset/NL2SparQL4NLU.test_no_stop_word.utterances.txt",
                "dataset/NL2SparQL4NLU.test_norm_all_words_no_stop_word.utterances.txt")
norm_data_input("dataset/NL2SparQL4NLU.train_no_stop_word.conll.txt",
                "dataset/NL2SparQL4NLU.train_norm_all_words_no_stop_word.conll.txt",
                file_type='conll')
norm_data_input("dataset/NL2SparQL4NLU.test_no_stop_word.conll.txt",
                "dataset/NL2SparQL4NLU.test_norm_all_words_no_stop_word.conll.txt",
                file_type='conll')


Preparing Input Symbol Tables (`isyms.txt`)

In [ ]:
%%bash
dpath='NL2SparQL4NLU/dataset/NL2SparQL4NLU'
dtype=''

cp $dpath.train$dtype.utterances.txt trn.txt
cp $dpath.test$dtype.utterances.txt tst.txt

cp $dpath.train$dtype.conll.txt trn.conll
cp $dpath.test$dtype.conll.txt tst.conll

In [ ]:
trn_data = read_corpus('trn.txt')
trn_lex = cutoff(trn_data)

with open('isyms.trn.txt', 'w') as f:
    f.write("\n".join(trn_lex) + "\n")

In [ ]:
%%bash
ngramsymbols isyms.trn.txt isyms.txt

In [ ]:
%%bash
farcompilestrings \
    --symbols=isyms.txt \
    --keep_symbols \
    --unknown_symbol='<UNK>' \
    trn.txt trn.far

farcompilestrings \
    --symbols=isyms.txt \
    --keep_symbols \
    --unknown_symbol='<UNK>' \
    tst.txt tst.far

Generating Output Symbol Table (`osyms.txt`)

In [ ]:
types = get_chunks('trn.conll')

with open('osyms.u.lst.txt', 'w') as f:
    f.write("O" + "\n")
    for c in sorted(list(types)):
        f.write("B-"+ c + "\n")
        f.write("I-"+ c + "\n")

In [ ]:
%%bash
ngramsymbols osyms.u.lst.txt osyms.txt

Create test set

In [ ]:
%%bash
wdir='wdir'
mkdir -p $wdir

farextract --filename_prefix="$wdir/" tst.far
cp $wdir/tst.txt-0001 sent.fsa

fstprint sent.fsa

Create training data for language model:
- If using data set without stop words, set variable `data_type` equal to `_no_stop_word` to avoid applying `cutoff` function (avoid losing tags with low occurrences in data set)

In [ ]:
data_type = ''

In [ ]:
trn = read_corpus_conll('trn.conll')
tags = get_column(trn, column=-1)

with open('trn.t.txt', 'w') as f:
    for s in tags:
        f.write(" ".join(s) + "\n")
        
tlex = cutoff(tags)
with open('osyms.t.lst.txt', 'w') as f:
    f.write("\n".join(tlex) + "\n")

In [ ]:
%%bash
# make symbol table
ngramsymbols osyms.t.lst.txt osyms.t.txt
# compile data into FAR again
farcompilestrings \
    --symbols=osyms.t.txt \
    --keep_symbols \
    --unknown_symbol='<UNK>' \
    trn.t.txt trn.t.far

- Let's train a unigram language model.

In [ ]:
%%bash
ngramcount --order=3 trn.t.far trn.t1.cnt
ngrammake trn.t1.cnt t1.lm
ngramprint --symbols=osyms.t.txt --negativelogs t1.lm t1.probs

- Let's create a new $\lambda_{W2T}$ (let's call it $\lambda_{W2T_{T}}$ for "tags"):
    - following the same procedure we followed for $\lambda_{W2T_{U}}$, but using:
        - as input symbol table (`isyms.txt`)
        - as output symbol table (`t.osyms.txt`)
    - allowing `<unk> <unk>` and *word*-`<unk>` arcs

In [ ]:
make_w2t_mle('t1.probs', out='t1_mle.txt')

In [ ]:
%%bash
fstcompile --isymbols=osyms.t.txt --osymbols=osyms.t.txt --keep_isymbols --keep_osymbols t1_mle.txt t1_mle.bin

In [ ]:
make_w2t('isyms.txt', 'osyms.t.txt', out='w2t_t.txt')

In [ ]:
%%bash
# Let's compile it
fstcompile \
    --isymbols=isyms.txt \
    --osymbols=osyms.t.txt \
    --keep_isymbols \
    --keep_osymbols \
    w2t_t.txt w2t_t.bin

fstinfo w2t_t.bin | head -n 8

In [ ]:
%%bash
fstcompose sent.fsa w2t_t.bin | fstcompose - t1.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

In [ ]:
%%bash
wdir='wdir'
farr=($(ls $wdir))

for f in ${farr[@]}
do
    fstcompose $wdir/$f w2t_t.bin | fstcompose - t1.lm |\
        fstshortestpath | fstrmepsilon | fsttopsort | fstprint --isymbols=isyms.txt
done > w2t_t.t1.out

In [ ]:
refs = read_corpus_conll('tst.conll')
hyps = read_fst4conll('w2t_t.t1.out')

results = evaluate(refs, hyps)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

- The model still has $F_1=0$, since `O` is the tag with highest prior.
- Observe the weights in the output

### 1.3.4. Exercises
- Compare sizes of $\lambda_{W2T_{U}}$ and $\lambda_{W2T_{T}}$ for `# of arcs`
- Unigram models & $\lambda_{W2T}$
    - Test pipeline: $\lambda = \lambda_{W} \circ \lambda_{W2T_{U}} \circ \lambda_{LM_{1}}$


- Bigram models: train a *tag* bigram model (let's call it $\lambda_{LM_{2}}$)

    - Test pipeline with $\lambda_{W2T_{U}}$: $\lambda = \lambda_{W} \circ \lambda_{W2T_{U}} \circ \lambda_{LM_{2}}$
    - Test pipeline with $\lambda_{W2T_{T}}$: $\lambda = \lambda_{W} \circ \lambda_{W2T_{T}} \circ \lambda_{LM_{2}}$

### 1.3.5. Maximum Likelihood Estimation (Emission Probabilities)
- So far we haven't explored the relation between input and output
- The next thing we can do is to expose our model to observations and estimate $p(w_{i}|t_{i})$ from data.
- We can use `ngramcount` and `ngrammake` to make a smoothed probability model (we are using default, i.e. no parameters). 
- We need to estimate probabilities like we would estimate bigram probabilities, thus:
    - prepare lexicon with *tags* and *words*
    - read CoNLL format corpus into far (token per line, preprocessed)
    - count bigrams
    - make a bigram language model
    - print bigrams with weights (negative log probabilities)
    - choose bigrams (it will contain unigrams, as well as `<s>` and `</s>` bigrams)
    - convert to FST & compile
    
- Let's call the model $\lambda_{W2T_{MLE}}$

In [ ]:
%%bash
# lets use our symbol tables (since they both have been applied cut-off)
cat isyms.txt osyms.t.txt | cut -f 1 | sort | uniq > msyms.m.lst.txt
ngramsymbols msyms.m.lst.txt msyms.t.txt

# let's convert data to ngrams
cat trn.conll | sed '/^$/d' | awk '{print $2,$1}' > trn.w2t.txt

# compile to far
farcompilestrings \
    --symbols=msyms.t.txt \
    --keep_symbols \
        --unknown_symbol='<UNK>' \
    trn.w2t.txt trn.w2t.far
    
# count bigrams
ngramcount --order=2 trn.w2t.far trn.w2t.cnt
# make a model
ngrammake trn.w2t.cnt trn.w2t.lm

# print ngram probabilities as negative logs
ngramprint \
    --symbols=msyms.t.txt\
    --negativelogs \
    trn.w2t.lm trn.w2t.probs

- Let's define a python function to convert probabilities printout to W2T FST

In [ ]:
make_w2t_mle('trn.w2t.probs', out='trn.w2t_mle.txt')

In [ ]:
%%bash
fstcompile \
    --isymbols=osyms.t.txt \
    --osymbols=isyms.txt \
    --keep_isymbols \
    --keep_osymbols \
    trn.w2t_mle.txt w2t_mle.bin
    
# we need to invert it to have words on input
fstinvert w2t_mle.bin w2t_mle.inv.bin

fstinfo w2t_mle.inv.bin | head -n 8

#### Testing
Let's test it:

In [ ]:
%%bash
# fstcompose sent.fsa w2t_mle.inv.bin | fstprint
fstprint sent.fsa

In [ ]:
%%bash
fstcompose sent.fsa w2t_mle.inv.bin | fstprint

In [ ]:
%%bash
wdir='wdir'
farr=($(ls $wdir))

for f in ${farr[@]}
do
#  fstcompose $wdir/$f w2t_mle.inv.bin | fstshortestpath | fstrmepsilon | fsttopsort | fstprint
  fstcompose $wdir/$f w2t_mle.inv.bin | fstcompose - t1.lm |  fstshortestpath | fstrmepsilon | fsttopsort | fstprint
#  fstcompose $wdir/$f w2t_mle.inv.bin | fstcompose - t1_mle.bin | fstshortestpath | fstrmepsilon | fsttopsort | fstprint
done > w2t_t.t1.mle_full.out

In [ ]:
refs = read_corpus_conll('tst.conll')
hyps = read_fst4conll('w2t_t.t1.mle_full.out')

results = evaluate(refs, hyps)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

- The pipeline above represents 

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)}$$

- To extend it to unigram tagging model we need to compose it with  $\lambda_{LM_{1}} = p(t_i)$ 

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{MLE}} \circ \lambda_{LM_{1}}$$

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i)}$$ 

In [ ]:
%%bash
fstcompose sent.fsa w2t_mle.inv.bin | fstcompose - t1.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

#### Exercise 1: Maximum Likelihood Estimation
- using `ngramprint` verify the Maximum Likelihood Estimation method (without `--negativelogs` it prints raw probabilities)
    - print bigram counts from $\lambda_{W2T_{MLE}}$ (output of `ngramcount`)
    - print unigram counts for either from $\lambda_{LM_{1}}$ or $\lambda_{W2T_{MLE}}$ (output of `ngramcount`)
    - using these counts compute probability of $p($ `brad|B-actor.name` $)$
    - extract probability of $p($ `brad|B-actor.name` $)$ from $\lambda_{W2T_{MLE}}$ (output of `ngrammake`)
    - compare values
    - repeat the procedure using counts from methods developed for the lab on ngram modeling.

#### Exercise 2: Markov Model Tagger
- Evaluate the MLE pipeline using bigram model on tags, i.e.

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{MLE}} \circ \lambda_{LM_{2}}$$

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i|t_{i-1})}$$ 

- compare performances to the HMM tagger from previous lab (NLTK)

## 2. Joint Distribution Modeling

As we have seen, sequence labeling for Language Understanding could be approached using Hidden Markov Models (similar to Part-of-Speech Tagging), and to models it as in the table below (__HMM__). Stochastic Conceptual Language Models for Spoken Language Understanding in [Raymond & Riccardi (2007)](https://disi.unitn.it/~riccardi/papers2/IS07-GenerDiscrSLU.pdf) (__R&R__) model it jointly.


| Model   | Equation |
|:--------|:----------
| __HMM__ | $$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i|t_{i-N+1}^{i-1})}$$
| __R&R__ | $$p(w_{1}^n,t_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_{i}t_{i}|w_{i-N+1}^{i-1}t_{i-N+1}^{i-1})}$$


From implementation perspective, joint modeling implies the following:
- we need to train $\lambda_{SCLM}$ on word-tag pairs
    - create corpus in a format for estimating $p(w_i,t_i|w_{i-N+1}^{i-1}t_{i-N+1}^{i-1})$
    - create symbol tables
- we need to change $\lambda_{W2T}$ to output *word-tag* pairs (let's call it $\lambda_{W2WT}$)
    - create FST like above for $\lambda_{W2WT}$ ($\lambda_{W2WT_{WT}}$ - to differentiate from $\lambda_{W2WT_{U}}$ that contains all possible combinations)
- we also need to change our input symbol table to accommodate OOV words due to joint modeling

#### Preparing Symbol Tables
- Let's create output symbol table the same way we did for $\lambda_{W2T_{T}}$
- Let's create input symbol taking $w$ from the $w,t$ pair

In [ ]:
# create training data in utterance-per-line format for output symbols (w+t)
trn = read_corpus_conll('trn.conll')
wt_sents = [["+".join(w) for w in s] for s in trn]
wt_osyms = cutoff(wt_sents)
wt_isyms = [w.split('+')[0] for w in wt_osyms]

with open('trn.wt.txt', 'w') as f:
    for s in wt_sents:
        f.write(" ".join(s) + "\n")
        
with open('osyms.wt.lst.txt', 'w') as f:
    f.write("\n".join(wt_osyms) + "\n")
    
with open('isyms.wt.lst.txt', 'w') as f:
    f.write("\n".join(wt_isyms) + "\n")

In [ ]:
%%bash
ngramsymbols osyms.wt.lst.txt osyms.wt.txt
ngramsymbols isyms.wt.lst.txt isyms.wt.txt

- Let's:
    - compile our processed data into FAR
    - train ngram language models on it - $\lambda_{SCLM}$

#### Training Conceptual Language Model

In [ ]:
%%bash
# compile data into FAR
farcompilestrings \
    --symbols=osyms.wt.txt \
    --keep_symbols \
    --unknown_symbol='<UNK>' \
    trn.wt.txt trn.wt.far

# train ngram model
ngramcount --order=2 trn.wt.far trn.wt.cnt
ngrammake trn.wt.cnt wt2.lm
ngraminfo wt2.lm

#### Building W2WT FST

- Let's build unweighted $\lambda_{W2WT_{WT}}$, using
    - input symbol table `isyms.wt.txt`
    - output symbol table `osyms.wt.txt`

In [ ]:
def make_w2t_wt(isyms, sep='+', out='w2wt.tmp'):
    special = {'<epsilon>', '<s>', '</s>'}
    oov = '<unk>'  # unknown symbol
    state = '0'    # wfst specification state
    fs = " "       # wfst specification column separator
    
    ist = sorted(list(set([line.strip().split("\t")[0] for line in open(isyms, 'r')]) - special))
    
    with open(out, 'w') as f:
        for e in ist:
            f.write(fs.join([state, state, e.split('+')[0], e]) + "\n")
        f.write(state + "\n")

In [ ]:
make_w2t_wt('osyms.wt.txt', out='w2wt_wt.txt')

In [ ]:
%%bash
# Let's compile it
fstcompile \
    --isymbols=isyms.wt.txt \
    --osymbols=osyms.wt.txt \
    --keep_isymbols \
    --keep_osymbols \
    w2wt_wt.txt w2wt_wt.bin

fstinfo w2wt_wt.bin | head -n 8

- Lets test the whole $\lambda_{W} \circ \lambda_{W2WT_{WT}} \circ \lambda_{SCLM_{2}}$

#### Preparing Test Data
- since we have changed input symbol table we need to recompile & extract our test data

In [ ]:
%%bash
farcompilestrings \
    --symbols=isyms.wt.txt \
    --keep_symbols \
    --unknown_symbol='<UNK>' \
    tst.txt tst.wt.far

wdir='wdir_wt'
mkdir -p $wdir

farextract --filename_prefix="$wdir/" tst.wt.far
cp $wdir/tst.txt-0001 sent.wt.fsa

fstprint sent.wt.fsa

In [ ]:
%%bash
fstcompose sent.wt.fsa w2wt_wt.bin | fstcompose - wt2.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

#### Evaluation
- Since on the output we have `word+tag`, we need to post-process the output for evaluation
- the function `read_fst4conll` already has that functionality via `split=True` and `sep='+'`

In [ ]:
%%bash
wdir='wdir_wt'
farr=($(ls $wdir))

for f in ${farr[@]}
do
    fstcompose $wdir/$f w2wt_wt.bin | fstcompose - wt2.lm |\
        fstshortestpath | fstrmepsilon | fsttopsort | fstprint --isymbols=isyms.wt.txt
done > w2wt_wt.wt2.out

In [ ]:
refs = read_corpus_conll('tst.conll')
hyps = read_fst4conll('w2wt_wt.wt2.out', split=True)

results = evaluate(refs, hyps)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

#### Exercise: Full $\lambda_{W2WT_{U}}$
- Implement $\lambda_{W2WT_{U}}$ using 'full' input and output symbol tables (`isyms.txt` and `osyms.txt`)
- Test the pipeline: $\lambda_{W} \circ \lambda_{W2WT_{U}} \circ \lambda_{SCLM_{2}}$
    - Observe the issues

#### Exercise
- Compare each pipeline in terms of:
    - size of input symbol table
    - size of output symbol table
    - size (number of arcs) of $\lambda_{W2T}$
    - size of $\lambda_{*LM}$

## 3. Common Improvements

- Training an ngram language model on data that contains tags only (i.e. $\lambda_{*LM}$) has one __big issue__: the out-of-span tag (`'O'`) is very frequent, consequently, there is not enough context to learn a good ngram model. 
- Joint modeling of words and tags, i.e. $\lambda_{SCLM}$, on the other hand, has a very specific (and less frequent context).
- There are two common enhancements to these models:
    - removing out-of-span tag `'O'` from the $\lambda_{*LM}$ to provide context for other tags
    - generalization of input into __classes__, i.e. $\lambda_{G}$, so that the data is less sparse

### Input Generalization (Normalization)

Th Language Understanding pipeline (as presented during the lectures) is 

$$\lambda = \lambda_{W} \circ \lambda_{G} \circ \lambda_{W2T} \circ \lambda_{*LM}$$

The function of $\lambda_{G}$ is this pipeline is to *generalize* the input, reducing sparsity.

#### Normalization
In Natural Language Processing it is common to __normalize__ (pre-process) the input data to reduce sparsity (e.g. [textacy](https://chartbeat-labs.github.io/textacy/build/html/api_reference/text_processing.html))'s pre-processing). 

The __normalization__ replaced all members of the __infinite set__ with a single __unique token__ with respect to a __common pattern__. It is not possible to learn a good model for each possible number, for instance.

- The example "entities" that have common pattern are:
    - numbers
    - emails
    - url
    - phone numbers
    - credit card numbers
    - etc.

These "entities" are generally captured using __regular expressions__.

#### Lookup Tables
Lookup tables provide a convenient way to generalize members of __large__ and __known set__ of entities. The common examples are *cities*, *countries*, *airport codes*, *movie names*, etc.
Even though these sets are potentially infinite, the lists of cities and movie names are generally available as external __Knowledge Bases__, and it is possible to check membership of a token.

#### [(Named) Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition)
> Named-entity recognition (NER) (also known as entity identification, entity chunking and entity extraction) is a subtask of information extraction that seeks to locate and classify named entity mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

__NER__ also covers entities that are covered by *normalization*, as it is a matter of approach (regex vs. sequence labeling).
There are several NLP tasks that fall under this category, specified with respect to the type of entity:
    - TIMEX - temporal expressions
    - ENAMEX - named entities 
    - NUMEX - numerical expressions
    - etc. (e.g. protein names in BioMedical Domain)

The task is similar to Concept Tagging, with an __important__ differences: 
- the same entity (from NER perspective) may belong to different classes in the target domain: 
    - e.g. in `NL2SparQL4NLU`: `actor.name`, `producer.name`, `director.name` are subclasses of a `PERSON`

Consequently, the output of such systems could be used as input for Concept Tagging.

#### Exercise: Lab
- Implement number generalization to map all numerical expressions in input to `<num>`
- Evaluate the pipeline with this step
- Observe sizes of input and output symbol tables